In [1]:
%pylab inline

import numpy as np
np.random.seed(666)

import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
%run -i examples/runDipoleFitterOnThreeImages.py

/Users/dreiss/GIT_REPOS/ip_diffim/tests/testDipoleFitter.py
26 Box2I((603, 195), (628, 221)) 438 2
   FOOTPRINT CENTER: 209 613 44.5836105347
   FOOTPRINT CENTER: 205 617 -38.0424880981
29 Box2I((1077, 263), (1113, 291)) 616 3
   FOOTPRINT CENTER: 276 1092 25.1447162628
   FOOTPRINT CENTER: 283 1105 7.36927890778
   FOOTPRINT CENTER: 271 1097 -14.904384613


/Users/dreiss/lsstsw/miniconda/lib/python2.7/site-packages/lmfit-0.9.2-py2.7.egg/lmfit/model.py:459: UserWarning: The keyword argument footprint does notmatch any arguments of the model function.It will be ignored.
/Users/dreiss/lsstsw/miniconda/lib/python2.7/site-packages/lmfit-0.9.2-py2.7.egg/lmfit/model.py:459: UserWarning: The keyword argument rel_weight does notmatch any arguments of the model function.It will be ignored.
/Users/dreiss/lsstsw/miniconda/lib/python2.7/site-packages/lmfit-0.9.2-py2.7.egg/lmfit/model.py:459: UserWarning: The keyword argument psf does notmatch any arguments of the model function.It will be ignored.


30 Box2I((364, 288), (378, 301)) 119 2
   FOOTPRINT CENTER: 254 1318 -6.08423614502
   FOOTPRINT CENTER: 294 371 -6.26749229431


LengthError: 
  File "src/image/Image.cc", line 92, in static _view_t lsst::afw::image::ImageBase<float>::_makeSubView(const geom::Extent2I &, const geom::Extent2I &, const _view_t &) [PixelT = float]
    Box2I(Point2I(-1304,-240),Extent2I(0,0)) doesn't fit in image 29x29 {0}
lsst::pex::exceptions::LengthError: 'Box2I(Point2I(-1304,-240),Extent2I(0,0)) doesn't fit in image 29x29'


In [ ]:
i = 115; s = catalog[i]
i = 12; s = catalog[i]
fp = s.getFootprint()
print i, fp.getBBox(), fp.getNpix(), len(fp.getPeaks())
for pk in fp.getPeaks():
    print '   FOOTPRINT CENTER:', pk.getIy(), pk.getIx(), pk.getPeakValue()
    
bbox = fp.getBBox()
subim = afwImage.ImageF(diffim.getMaskedImage().getImage(), bbox, afwImage.PARENT)
dft.DipoleUtils.displayImage(subim)
subim = afwImage.ImageF(posImage.getMaskedImage().getImage(), bbox, afwImage.PARENT)
dft.DipoleUtils.plt.figure(figsize=(8, 2.5)); dft.DipoleUtils.plt.subplot(1, 3, 1)
dft.DipoleUtils.display2dArray(np.log10(subim.getArray()+50.))
subim = afwImage.ImageF(negImage.getMaskedImage().getImage(), bbox, afwImage.PARENT)
dft.DipoleUtils.plt.figure(figsize=(8, 2.5)); dft.DipoleUtils.plt.subplot(1, 3, 1)
dft.DipoleUtils.display2dArray(np.log10(subim.getArray()+50.))

subim = afwImage.ImageF(posImage.getMaskedImage().getImage(), bbox, afwImage.PARENT)
subArr = subim.getArray()
print np.median(subArr), np.max(subArr)
psfImg = posImage.getPsf().computeImage()
pkToFlux = np.nansum(psfImg.getArray()) / posImage.getPsf().computePeak()
print pkToFlux
print np.max(subArr) * pkToFlux

result = dft.DipoleFitAlgorithm.fitDipole_new(
    diffim, s, posImage, negImage, rel_weight=0.5, fitBgGradient=False, separateNegParams=False,
    verbose=False, display=True, return_fitObj=True)

result

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import lmfit
result0 = result[1]
print result0.fit_report()

In [ ]:
bbox = fp.getBBox()
difSubim = afwImage.ImageF(diffim.getMaskedImage().getImage(), bbox, afwImage.PARENT)
posSubim = afwImage.ImageF(posImage.getMaskedImage().getImage(), bbox, afwImage.PARENT)
negSubim = afwImage.ImageF(negImage.getMaskedImage().getImage(), bbox, afwImage.PARENT)

plt.plot(np.log10(posSubim.getArray().flatten()), np.log10(negSubim.getArray().flatten()), 'o')

PSFs are identical between images...

In [ ]:
dif_p = diffim.getPsf().computeImage()
#dft.DipoleUtils.displayImage(dif_p)
pos_p = posImage.getPsf().computeImage()
#dft.DipoleUtils.displayImage(pos_p)
neg_p = negImage.getPsf().computeImage()
#dft.DipoleUtils.displayImage(neg_p)

tmp = pos_p.clone()
tmp -= dif_p
dft.DipoleUtils.displayImage(tmp)

Here is a weird case - a bright galaxy with a dipole in the core.

In [ ]:
i = 115; s = catalog[i]
#i = 12; s = catalog[i]
fp = s.getFootprint()
print i, fp.getBBox(), fp.getNpix(), len(fp.getPeaks())
for pk in fp.getPeaks():
    print '   FOOTPRINT CENTER:', pk.getIy(), pk.getIx(), pk.getPeakValue()
    
bbox = fp.getBBox()
subim = afwImage.ImageF(diffim.getMaskedImage().getImage(), bbox, afwImage.PARENT)
dft.DipoleUtils.displayImage(subim)
subim = afwImage.ImageF(posImage.getMaskedImage().getImage(), bbox, afwImage.PARENT)
dft.DipoleUtils.plt.figure(figsize=(8, 2.5)); dft.DipoleUtils.plt.subplot(1, 3, 1)
dft.DipoleUtils.display2dArray(np.log10(subim.getArray()))
subim = afwImage.ImageF(negImage.getMaskedImage().getImage(), bbox, afwImage.PARENT)
dft.DipoleUtils.plt.figure(figsize=(8, 2.5)); dft.DipoleUtils.plt.subplot(1, 3, 1)
dft.DipoleUtils.display2dArray(np.log10(subim.getArray()))

subim = afwImage.ImageF(posImage.getMaskedImage().getImage(), bbox, afwImage.PARENT)
subArr = subim.getArray()
print np.median(subArr), np.max(subArr)
psfImg = posImage.getPsf().computeImage()
pkToFlux = np.nansum(psfImg.getArray()) / posImage.getPsf().computePeak()
print pkToFlux
print np.max(subArr) * pkToFlux

result = dft.DipoleFitAlgorithm.fitDipole_new(
    diffim, s, posImage, negImage, rel_weight=1., fitBgGradient=True, separateNegParams=False,
    verbose=False, display=True, return_fitObj=True)

result

In [ ]:
result0 = result[1]
print result0.fit_report()

In [ ]:
i = 115; s = catalog[i]
i = 67; s = catalog[i]
#i = 12; s = catalog[i]
fp = s.getFootprint()
print i, fp.getBBox(), fp.getNpix(), len(fp.getPeaks())
for pk in fp.getPeaks():
    print '   FOOTPRINT CENTER:', pk.getIy(), pk.getIx(), pk.getPeakValue()
    
bbox = fp.getBBox()
subim = afwImage.ImageF(diffim.getMaskedImage().getImage(), bbox, afwImage.PARENT)
dft.DipoleUtils.displayImage(subim)
subim = afwImage.ImageF(posImage.getMaskedImage().getImage(), bbox, afwImage.PARENT)
dft.DipoleUtils.plt.figure(figsize=(8, 2.5)); dft.DipoleUtils.plt.subplot(1, 3, 1)
dft.DipoleUtils.display2dArray(np.log10(subim.getArray()+20.))
subim = afwImage.ImageF(negImage.getMaskedImage().getImage(), bbox, afwImage.PARENT)
dft.DipoleUtils.plt.figure(figsize=(8, 2.5)); dft.DipoleUtils.plt.subplot(1, 3, 1)
dft.DipoleUtils.display2dArray(np.log10(subim.getArray()))

subim = afwImage.ImageF(posImage.getMaskedImage().getImage(), bbox, afwImage.PARENT)
subArr = subim.getArray()
print np.median(subArr), np.max(subArr)
psfImg = posImage.getPsf().computeImage()
pkToFlux = np.nansum(psfImg.getArray()) / posImage.getPsf().computePeak()
print pkToFlux
print np.max(subArr) * pkToFlux

result = dft.DipoleFitAlgorithm.fitDipole_new(
    diffim, s, posImage, negImage, rel_weight=1., fitBgGradient=True, separateNegParams=False,
    verbose=False, display=True, return_fitObj=True)

result